In [18]:
import sys
sys.path.append("..")
from lib import *
from sklearn.utils import shuffle

In [2]:
%mkdir -p data/research

In [3]:
import urllib.request as request
file_path = 'data/research/sonar.all-data.csv'
d_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/connectionist-bench/sonar/sonar.all-data'
request.urlretrieve(d_url, file_path)

('data/research/sonar.all-data.csv', <http.client.HTTPMessage at 0x150f09e320>)

In [51]:
df = pd.read_csv(file_path, header=None)
df = shuffle(df)
df = df.reset_index()
depth = 3
benchmark = None
rows = list(range(1, 170))
features = list(range(3, 10))

In [52]:
df.head()

,index,0,1,2,3,4,5,6,7,8,...,51,52,53,54,55,56,57,58,59,60
0,107,0.0428,0.0555,0.0708,0.0618,0.1215,0.1524,0.1543,0.0391,0.0610,...,0.0142,0.0179,0.0079,0.0060,0.0131,0.0089,0.0084,0.0113,0.0049,M
1,155,0.0211,0.0128,0.0015,0.0450,0.0711,0.1563,0.1518,0.1206,0.1666,...,0.0117,0.0023,0.0047,0.0049,0.0031,0.0024,0.0039,0.0051,0.0015,M
2,105,0.0116,0.0179,0.0449,0.1096,0.1913,0.0924,0.0761,0.1092,0.0757,...,0.0163,0.0099,0.0084,0.0270,0.0277,0.0097,0.0054,0.0148,0.0092,M
3,17,0.0192,0.0607,0.0378,0.0774,0.1388,0.0809,0.0568,0.0219,0.1037,...,0.0331,0.0131,0.0120,0.0108,0.0024,0.0045,0.0037,0.0112,0.0075,R
4,200,0.0131,0.0387,0.0329,0.0078,0.0721,0.1341,0.1626,0.1902,0.2610,...,0.0150,0.0076,0.0032,0.0037,0.0071,0.0040,0.0009,0.0015,0.0085,M


In [54]:
# print(df[:100])
f1 = RNF(df.loc[:110], 3, 3, 1, 40, 20)
f1.fit()
f1.predict(df[150:160])


# t1 = Tree(shuffle(df), depth, benchmark, rows, features)
# t1.fit()
# # print(t1.calc_oob_error())
# t1.calc_oob_error()
# # t1.predict(df.loc[[1]])

Head is a pure node.
Head is a pure node.


([array([ 0.6,  0.4]),
  array([ 0.26666667,  0.73333333]),
  array([ 0.6,  0.4]),
  array([ 0.6,  0.4]),
  array([ 0.26666667,  0.73333333]),
  array([ 0.6,  0.4]),
  array([ 0.26666667,  0.73333333]),
  array([ 0.26666667,  0.73333333]),
  array([ 0.26666667,  0.73333333]),
  array([ 0.6,  0.4])],
 ['R', 'M', 'R', 'R', 'M', 'R', 'M', 'M', 'M', 'R'])

In [55]:
f1.update(df[:120])
f1.predict(df[150:160])

NameError: name 'trees' is not defined

In [16]:
print(df)

         0       1       2       3       4       5       6       7       8   \
0    0.0200  0.0371  0.0428  0.0207  0.0954  0.0986  0.1539  0.1601  0.3109   
1    0.0453  0.0523  0.0843  0.0689  0.1183  0.2583  0.2156  0.3481  0.3337   
2    0.0262  0.0582  0.1099  0.1083  0.0974  0.2280  0.2431  0.3771  0.5598   
3    0.0100  0.0171  0.0623  0.0205  0.0205  0.0368  0.1098  0.1276  0.0598   
4    0.0762  0.0666  0.0481  0.0394  0.0590  0.0649  0.1209  0.2467  0.3564   
5    0.0286  0.0453  0.0277  0.0174  0.0384  0.0990  0.1201  0.1833  0.2105   
6    0.0317  0.0956  0.1321  0.1408  0.1674  0.1710  0.0731  0.1401  0.2083   
7    0.0519  0.0548  0.0842  0.0319  0.1158  0.0922  0.1027  0.0613  0.1465   
8    0.0223  0.0375  0.0484  0.0475  0.0647  0.0591  0.0753  0.0098  0.0684   
9    0.0164  0.0173  0.0347  0.0070  0.0187  0.0671  0.1056  0.0697  0.0962   
10   0.0039  0.0063  0.0152  0.0336  0.0310  0.0284  0.0396  0.0272  0.0323   
11   0.0123  0.0309  0.0169  0.0313  0.0358  0.0102 